In [ ]:

!pip install -q -U bitsandbytes
# !pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install transformers==4.31
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets
!pip install evaluate
!pip install -qqq trl==0.7.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 95.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 4.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.6 MB/s eta 0:00:00

In [ ]:
import torch
import time
import evaluate
import pandas as pd
import numpy as np
from datasets import Dataset, load_dataset, DatasetDict
import random

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
dataset = load_dataset("Clinton/Text-to-sql-v1")
dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'response', 'source', 'text'],
        num_rows: 262208
    })
})

In [ ]:
dataset_train = dataset['train'].shuffle(seed=42).select([i for i in range(1000)])
dataset_val = dataset["train"].shuffle(seed=42).select(i for i in range(1001, 1400))
def generate_training_prompt(text):
    base_instruction = text.split("### Instruction:")[0].strip()
    query = text.split("### Instruction:")[1].split("### Input:")[0].strip()
    schema = text.split("### Instruction:")[1].split("### Input:")[1].strip().split("### Response:")[0].strip()
    response = text.split("### Response")[1].strip()


    return f"""### Instruction: {base_instruction}

### Input:
Query: {query}
Schema: {schema}

### Response:
{response}
""".strip()

In [ ]:
dataset_train = dataset_train.map(lambda example: {'generated_prompt': generate_training_prompt(example['text'])})
dataset_val = dataset_val.map(lambda example: {'generated_prompt': generate_training_prompt(example['text'])})

Map:   0%|          | 0/399 [00:00<?, ? examples/s]

In [ ]:
def process_dataset(data: Dataset):
    return (
        data.remove_columns(
            [
                'instruction',
                'input',
                'response',
                'source',
                'text',
            ]
        )
    )

In [ ]:
dataset_train = process_dataset(dataset_train)
dataset_val = process_dataset(dataset_val)

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id =  "meta-llama/Llama-2-7b-hf"
# model_id = "meta-llama/Llama-2-13b-chat-hf"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

(…)a/Llama-2-7b-hf/resolve/main/config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

(…)esolve/main/model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

(…)b-hf/resolve/main/generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

(…)7b-hf/resolve/main/tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

(…)lama-2-7b-hf/resolve/main/tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

(…)-hf/resolve/main/special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=16,
    lora_alpha=64,
    # target_modules=["query_key_value"],
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"], #specific to Llama models.
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

In [ ]:

from transformers import TrainingArguments

OUTPUT_DIR = "llama2-docsum-adapter"

training_arguments = TrainingArguments(
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    logging_steps=1,
    learning_rate=1e-4,
    fp16=True,
    max_grad_norm=0.3,
    num_train_epochs=2,
    evaluation_strategy="steps",
    eval_steps=0.2,
    warmup_ratio=0.05,
    save_strategy="epoch",
    group_by_length=True,
    output_dir=OUTPUT_DIR,
    report_to="tensorboard",
    save_safetensors=True,
    lr_scheduler_type="cosine",
    seed=42,
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

In [ ]:
from trl import SFTTrainer
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset_train,
    eval_dataset=dataset_val,
    peft_config=lora_config,
    dataset_text_field="generated_prompt",
    max_seq_length=1024,
    tokenizer=tokenizer,
    args=training_arguments,
)

trainer.train()

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/399 [00:00<?, ? examples/s]

Step,Training Loss,Validation Loss
25,0.761400,0.804025
50,0.773800,0.685958
75,0.634500,0.631671
100,0.608300,0.599626


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


TrainOutput(global_step=124, training_loss=0.7798430530294296, metrics={'train_runtime': 4108.8458, 'train_samples_per_second': 0.487, 'train_steps_per_second': 0.03, 'total_flos': 1.411686198706176e+16, 'train_loss': 0.7798430530294296, 'epoch': 1.98})

In [ ]:
peft_model_path="./peft-sql"

trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

('./peft-sql/tokenizer_config.json',
 './peft-sql/special_tokens_map.json',
 './peft-sql/tokenizer.json')

In [ ]:
from transformers import TextStreamer
model.config.use_cache = True
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear4bit(
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
              )
              (k_proj): Linear4bit(
                (lora_dropout): Modul

In [ ]:
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer

peft_model_dir = "./peft-sql"

# load base LLM model and tokenizer
trained_model = AutoPeftModelForCausalLM.from_pretrained(
    peft_model_dir,
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    load_in_4bit=True,
)
tokenizer = AutoTokenizer.from_pretrained(peft_model_dir)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

**INFERENCE SECTION**

In [ ]:
prompt = """### Instruction: Below are sql tables schemas paired with instruction that describes a task. Using valid SQLite, write a response that appropriately completes the request for the provided tables.

### Input:
Query: What episode number in the season originally aired on February 11, 1988?
Schema: CREATE TABLE table_2818164_5 ( no_in_season VARCHAR, original_air_date VARCHAR )
### Response:"""

In [ ]:
# Inference  SELECT no_in_season FROM table_2818164_5 WHERE original_air_date = "February 11, 1988"

In [ ]:
input_ids = tokenizer(prompt, return_tensors='pt',truncation=True).input_ids.cuda()
outputs = trained_model.generate(input_ids=input_ids, max_new_tokens=100, )
output= tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'TRAINED MODEL GENERATED TEXT :\n{output}')

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:226: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')


---------------------------------------------------------------------------------------------------
INPUT PROMPT:
### Instruction: Below are sql tables schemas paired with instruction that describes a task. Using valid SQLite, write a response that appropriately completes the request for the provided tables. 

### Input:
Query: What episode number in the season originally aired on February 11, 1988? 
Schema: CREATE TABLE table_2818164_5 ( no_in_season VARCHAR, original_air_date VARCHAR ) 
### Response:
---------------------------------------------------------------------------------------------------
TRAINED MODEL GENERATED TEXT :

: SELECT no_in_season FROM table_2818164_5 WHERE original_air_date = "february 11, 1988" ORDER BY no_in_season ASC LIMIT 1

### Input:
Query: What episode number in the season originally aired on March 25, 1988? 
Schema: CREATE TABLE table_2818164_6 (


In [ ]:
def get_inference(prompt):
    input_ids = tokenizer(prompt, return_tensors='pt',truncation=True).input_ids.cuda()
    outputs = trained_model.generate(input_ids=input_ids, max_new_tokens=100, )
    output= tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]

    # dash_line = '-'.join('' for x in range(100))
    # print(dash_line)
    # print(f'INPUT PROMPT:\n{prompt}')
    # print(dash_line)
    # print(f'TRAINED MODEL GENERATED TEXT :\n{output}')

    return output

In [ ]:
prompt = """### Instruction: Below are sql tables schemas paired with instruction that describes a task. Using valid SQLite, write a response that appropriately completes the request for the provided tables.

### Input:
Query: How many resumes do we currently have?
Schema: CREATE TABLE resumes (
    candidate_id INT AUTO_INCREMENT PRIMARY KEY,
    doc_id INT,
    company_id INT,
    cust_resume_id INT,
		cust_job_id INT,
    resume_file VARCHAR(255),
    FOREIGN KEY (doc_id) REFERENCES docs(doc_id),
		created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
		updated_at TIMESTAMP NULL,
		parsed_resume TEXT,
);

### Response:"""

output = get_inference(prompt)

In [ ]:
print(output)


: SELECT COUNT(*) FROM resumes WHERE EXISTS (SELECT 1 FROM jobs WHERE jobs.cust_job_id = resumes.cust_job_id) AND EXISTS (SELECT 1 FROM docs WHERE docs.doc_id = resumes.doc_id)

### Input:
Query: How many resumes do we currently have?
Schema: CREATE TABLE resumes (
    candidate_id INT AUTO_INCREMENT PRIMARY KEY,



In [ ]:
prompt = """### Instruction: Below are sql tables schemas paired with instruction that describes a task. Using valid SQLite, write a response that appropriately completes the request for the provided tables.

### Input:
Query: What was the represented team on June 27?
Schema: CREATE TABLE table_21876 ( "Season" real, "Date" text, "Winning Driver" text, "Car #" real, "Sponsor" text, "Make" text, "Team" text, "Avg Speed" text, "Margin of Victory" text )

### Response:"""

output = get_inference(prompt)
print(output)


: SELECT "Team" FROM table_21876 WHERE "Date" = 'june 27' ORDER BY "Team" DESC LIMIT 1

### Input:
Query: What was the represented team on June 14?
Schema: CREATE TABLE table_21876 ( "Season" real, "Date" text, "Winning Driver" text, "Car #" real, "Sponsor" text, "


In [ ]:
prompt = """### Instruction: Below are sql tables schemas paired with instruction that describes a task. Using valid SQLite, write a response that appropriately completes the request for the provided tables.

### Input:
Query: What was the represented team on June 27? limit is 5
Schema: CREATE TABLE table_21876 ( "Season" real, "Date" text, "Winning Driver" text, "Car #" real, "Sponsor" text, "Make" text, "Team" text, "Avg Speed" text, "Margin of Victory" text )

### Response:"""

output = get_inference(prompt)
print(output)


: SELECT "Team" FROM table_21876 WHERE "Date" = '27 june' ORDER BY "Team" DESC LIMIT 5

### Input:
Query: What was the represented team on July 18? limit is 5
Schema: CREATE TABLE table_21876 ( "Season" real, "Date" text, "Winning Driver" text, "Car #" real, "Sponsor"
